In [ ]:
#r "nuget: Plotly.NET, 3.0.0"
#r "nuget: Plotly.NET.Interactive, 3.0.0"
#r "nuget: Microsoft.Data.Analysis, 0.18.0"

Installed Packages Microsoft.Data.Analysis, 0.18.0 Plotly.NET, 3.0.0 Plotly.NET.Interactive, 3.0.0

Loading extensions from `Microsoft.Data.Analysis.Interactive.dll`

Loading extensions from `Plotly.NET.Interactive.dll`

In [ ]:

open System.IO
open System.Diagnostics
open Microsoft.Data.Analysis
open Plotly.NET

let parseMilliseconds (str: string) =
    if str.Contains(" ms") then
        Single.Parse(str.Replace(" ms", ""))
    elif str.Contains(" s") then
        Single.Parse(str.Replace(" s", "")) * 1000.f
    else
        failwith "Invalid string"

let parseAllocated (str: string) =
    if str.Contains(" MB") then
        Single.Parse(str.Replace(" MB", ""))
    elif str.Contains(" KB") then
        Single.Parse(str.Replace(" KB", "")) / 1024.f
    else
        failwith "Invalid string"

let run name args workingDir =
    let info = ProcessStartInfo()
    info.WindowStyle <- ProcessWindowStyle.Hidden
    info.Arguments <- args
    info.FileName <- name
    info.UseShellExecute <- false
    info.WorkingDirectory <- workingDir
    info.RedirectStandardError <- true
    info.RedirectStandardOutput <- true
    info.RedirectStandardInput <- true
    info.CreateNoWindow <- true
    printfn $"Run {name} {args} in {workingDir}"
    let p = Process.Start(info)
    let o = p.StandardOutput.ReadToEnd()
    let errors = p.StandardError.ReadToEnd()
    p.WaitForExit()
    printfn "%A" o
    if p.ExitCode <> 0 then
        failwith $"Process {name} {args} failed: {errors}."


let resultsPath = Path.Combine(__SOURCE_DIRECTORY__, @"BenchmarkDotNet.Artifacts\results\BenchmarkComparison.TypeCheckingBenchmark-report.csv")

let makeMSBuildPropsArgs (props : (string * string) list) =
    props
    |> List.filter (fun (prop, value) -> String.IsNullOrWhiteSpace(value) |> not)
    |> List.map (fun (prop, value) -> $"/p:{prop}={value}")
    |> fun args -> String.Join(" ", args)

let runBenchmark (msBuildArgs : (string * string) list) =
    let msBuildArgsString = msBuildArgs |> makeMSBuildPropsArgs
    run "dotnet" $"build -c Release run.fsproj {msBuildArgsString}" __SOURCE_DIRECTORY__
    run "dotnet" $"run -c Release --no-build --project run.fsproj -- --filter *" __SOURCE_DIRECTORY__

let benchmarkCurrent(): string * DataFrame =
    printfn "Benchmarking Current (Today)..."
    let msBuildArgs =
        [
            "FcsReferenceType", "project"
            "FcsProjectPath", @"..\..\..\..\src\Compiler\FSharp.Compiler.Service.fsproj"
        ]
    runBenchmark msBuildArgs
    let df = DataFrame.LoadCsv(resultsPath)
    printfn "Current (Today) Done"
    ("Current (Today)", df)

let benchmarkVersion (name: string) (version: string) (constants: string): string * DataFrame =
    printfn $"Benchmarking {name}..."
    let msBuildArgs =
        [
            "FcsReferenceType", "nuget"
            "FcsNuGetVersion", version
            "DefineConstants", constants
        ]
    runBenchmark msBuildArgs

    let df = DataFrame.LoadCsv(resultsPath)
    printfn $"{name} Done"
    (name, df)

[<RequireQualifiedAccess>]
type UseVisualStudio =
    | Yes
    | No

let benchmarkCommit (commitHash: string) (constants: string) (useVisualStudio : UseVisualStudio): string * DataFrame =
    let tmp = Path.GetTempFileName()
    try File.Delete(tmp) with | _ -> ()
    let tmpDir = Path.Combine(Path.GetDirectoryName(tmp), Path.GetFileNameWithoutExtension(tmp))
    Directory.CreateDirectory(tmpDir) |> ignore

    try
        let fcsOutputPath = Path.Combine(tmpDir, "artifacts/bin/FSharp.Compiler.Service/Release/netstandard2.0/FSharp.Compiler.Service.dll")
        let msBuildArgs =
            [
                "FcsReferenceType", "dll"
                "FcsDllPath", fcsOutputPath
            ]
        printfn $"Cloning 'dotnet/fsharp.git' in '{tmpDir}'..."
        run "git" $"clone https://github.com/dotnet/fsharp.git {tmpDir}" __SOURCE_DIRECTORY__
        printfn $"Switching to '{commitHash}'..."
        run "git" $"reset --hard {commitHash}" tmpDir
        printfn "Building fsharp..."
        let vsArg = match useVisualStudio with | UseVisualStudio.Yes -> "" | UseVisualStudio.No -> " -noVisualStudio"
        run "cmd" $"/C build.cmd -c Release {vsArg}" tmpDir
        printfn $"Benchmarking {commitHash}..."
        runBenchmark msBuildArgs

        let df = DataFrame.LoadCsv(resultsPath)
        printfn $"{commitHash} Done"
        (commitHash, df)
    finally
        try Directory.Delete(tmpDir) with | _ -> ()

In [ ]:
let useVisualStudio = UseVisualStudio.No
let benchmarkData =
    [
      // Note: SERVICE_30_0_0 and SERVICE_13_0_0 refer to define constants in order to build the benchmark on older versions.
      benchmarkCurrent()
      //benchmarkCommit doesn't fully work yet
      benchmarkCommit "a901fe2862dce0644ac8104d24e51e664b2d553f" "" useVisualStudio
      //benchmarkCommit "81d1d918740e9ba3cb2eb063b6f28c3139ca9cfa" "" useVisualStudio
      //benchmarkCommit "1d36c75225436f8a7d30c4691f20d6118b657fec" "" useVisualStudio
      //benchmarkCommit "2e4096153972abedae142da85cac2ffbcf57fe0a" "" useVisualStudio
      //benchmarkCommit "af6ff33b5bc15951a6854bdf3b226db8f0e28b56" "" useVisualStudio
      // benchmarkVersion "40.0.0 (6/22/2021)" "40.0.0" ""
      // benchmarkVersion "35.0.0 (4/10/2020)" "35.0.0" "SERVICE_30_0_0"
      // benchmarkVersion "30.0.0 (6/29/2019)" "30.0.0" "SERVICE_30_0_0"
      // benchmarkVersion "25.0.1 (9/5/2018)" "25.0.1" "SERVICE_13_0_0"
      // benchmarkVersion "20.0.1 (2/21/2018)" "20.0.1" "SERVICE_13_0_0"
      // benchmarkVersion "13.0.0 (6/28/2017)" "13.0.0" "SERVICE_13_0_0"
    ]

Benchmarking Current (Today)...
Run dotnet build -c Release run.fsproj /p:FcsReferenceType=project /p:FcsProjectPath=..\..\..\..\src\Compiler\FSharp.Compiler.Service.fsproj in d:\projekty\fsharp\tests\benchmarks\FCSBenchmarks\BenchmarkComparison
"Microsoft (R) Build Engine version 17.2.0+41abc5629 for .NET
Copyright (C) Microsoft Corporation. All rights reserved.

  Determining projects to restore...
  Restored d:\projekty\fsharp\tests\benchmarks\FCSBenchmarks\BenchmarkComparison\run.fsproj (in 319 ms).
  Restored d:\projekty\fsharp\src\FSharp.DependencyManager.Nuget\FSharp.DependencyManager.Nuget.fsproj (in 319 ms).
  Restored d:\projekty\fsharp\src\Compiler\FSharp.Compiler.Service.fsproj (in 319 ms).
  Restored d:\projekty\fsharp\src\FSharp.Core\FSharp.Core.fsproj (in 318 ms).
  FSharp.Core -> d:\projekty\fsharp\artifacts\bin\FSharp.Core\Release\netstandard2.1\FSharp.Core.dll
  FSharp.Core -> d:\projekty\fsharp\artifacts\bin\FSharp.Core\Release\netstandard2.0\FSharp.Core.dll
  FSharp

In [ ]:
benchmarkData

index Item1 Item2 0 Current (Today) index Method Job AnalyzeLaunchVariance EvaluateOverhead MaxAbsoluteError MaxRelativeError MinInvokeCount MinIterationTime OutlierMode Affinity EnvironmentVariables Jit Platform PowerPlanMode Runtime AllowVeryLargeObjects Concurrent CpuGroups Force HeapAffinitizeMask HeapCount NoAffinitize RetainVm Server Arguments BuildConfiguration Clock EngineFactory NuGetReferences Toolchain IsMutator InvocationCount IterationCount IterationTime LaunchCount MaxIterationCount MaxWarmupIterationCount MemoryRandomization MinIterationCount MinWarmupIterationCount RunStrategy UnrollFactor WarmupCount Mean Error StdDev Gen 0 Gen 1 Gen 2 Allocated 0 Run Job-KXGNVC False Default Default Default Default Default Default 1.1111111E+15 Empty RyuJit X64 8c5e7fda-e8bf-4a96-9a85-a6e23a8c635c .NET 6.0 False True False True Default Default False False False Default Default Default Default Default Default Default 1 Default Default Default Default Default Default Default Default Default 1 Default 452.3 ms 9.02 ms 25.88 ms 25000 13000 6000 273 MB

In [ ]:
// Data cleanup

let df =
    (benchmarkData.[0] |> snd, benchmarkData.[1..])
    ||> List.fold (fun df (_, df2) ->
        df.Append(df2.Rows)
    )

let x =
    benchmarkData
    |> List.map (fun (name, _) -> name)

let meanColumn = df.Columns.["Mean"]
let allocatedColumn = df.Columns.["Allocated"]

let y1 =
    [
        for i = 0L to meanColumn.Length - 1L do
            meanColumn.[i] :?> string
            |> parseMilliseconds
    ]
let meanData = (x, y1) ||> List.zip |> List.rev
let y2 =
    [
        for i = 0L to allocatedColumn.Length - 1L do
            allocatedColumn.[i] :?> string
            |> parseAllocated
    ]
let allocatedData = (x, y2) ||> List.zip |> List.rev

// Charts

let meanLine = Chart.Line(meanData, Name="Mean (ms)")
let allocatedLine = Chart.Line(allocatedData, Name="Allocated (MB)")

Chart.combine([meanLine;allocatedLine])

<!-- Plotly chart will be drawn inside this DIV --> 


 
</div